In [5]:
!pip3 install tensorflow==2.12.0 --force-reinstall

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:02
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=2.0 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Obtaining dependency information for h5py>=2.9.0 from https://files.pythonhosted.o

  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.0.2
    Uninstalling setuptools-69.0.2:
      Successfully uninstalled setuptools-69.0.2
  Attempting uninstall: pyasn1
    Found existing installation: pyasn1 0.5.1
    Uninstalling pyasn1-0.5.1:
      Successfully uninstalled pyasn1-0.5.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.1
    Uninstalling protobuf-4.25.1:
      Successfully uninstalled protobuf-4.25.1
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packa

In [12]:
!pip3 install tensorflow-serving-api

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for tensorflow-serving-api from https://files.pythonhosted.org/packages/3c/4d/ce445324599c1a999ec9b78ad22f391ed91a4785522a7628ff8c99527a1d/tensorflow_serving_api-2.14.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tensorflow<3,>=2.14.1 from https://files.pythonhosted.org/packages/7c/95/b291edd75d76708f828039695b5561bdb6db1a95bca43f9087b794bd735a/tensorflow-2.15.0-cp39-cp39-macosx_10_15_x86_64.whl.metadata
  Obtaining dependency information for ml-dtypes~=0.2.0 from https://files.pythonhosted.org/packages/f0/d8/f602f05db13d187884ddb5ae4e823d333beb28bbd3d12c057450afa3acee/ml_dtypes-0.2.0-cp39-cp39-macosx_10_9_universal2.whl.metadata
  Obtaining dependency information for tensorboard<2.16,>=2.15 from https://files.pythonhosted.org/packages/6e/0c/1059a6682cf2cc1fcc0d5327837b5672fe4f5574255fa5430d0a8ceb75e9/tensorboard-2.15.1-py3-none-any.whl.metadata
  Us

In [ ]:
!pip3 install protobuf==3.20.3 --upgrade

In [16]:
!pip3 install keras-image-helper

Defaulting to user installation because normal site-packages is not writeable
  Using cached keras_image_helper-0.0.1-py3-none-any.whl (4.6 kB)

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [10]:
import tensorflow as tf
print(tf.version.VERSION)

/Users/balajig/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


2.12.0


In [9]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.getenv('PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION')

'python'

In [55]:
import grpc
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from keras_image_helper import create_preprocessor

In [56]:
def createRequest(url):
    X = preprocessor.from_url(url)
    x_proto = tf.make_tensor_proto(X, shape=X.shape)
    pbrequest = predict_pb2.PredictRequest()
    pbrequest.model_spec.name = 'sport-classification-model'
    pbrequest.inputs['input_7'].CopyFrom(x_proto)
    return pbrequest

def decodeResponse(pb_response):
    output = pb_response.outputs['dense_5'].float_val
    return labels[np.argmax(output)]


def decodeResponseDetail(pb_response):
    output = pb_response.outputs['dense_5'].float_val
    return dict(zip(labels, output))

In [58]:
preprocessor = create_preprocessor('inception_v3', target_size=(299,299))
host = "localhost:8500"
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
labels = ['Badminton', 'Cricket', 'Karate', 'Soccer', 'Swimming', 'Tennis', 'Wrestling']
url = 'https://i.pinimg.com/originals/50/c3/16/50c3163dfff9f2e7252f3d40e2d7f571.jpg'
pbrequest = createRequest(url)
pb_response = stub.Predict(pbrequest, timeout=10)
decodeResponseDetail(pb_response)



{'Badminton': 6.761700205970556e-05,
 'Cricket': 0.0013552604941651225,
 'Karate': 0.9984763264656067,
 'Soccer': 9.206405957229435e-05,
 'Swimming': 3.4510065916038e-07,
 'Tennis': 8.058004823396914e-06,
 'Wrestling': 3.975866320615751e-07}